# Análise exploratória de dados

A análise exploratória de dados é uma etapa fundamental para qualquer projeto que envolva dados. Ela consiste em examinar e estudar as características de um conjunto de dados, como sua distribuição, suas variáveis, suas relações e suas anomalias, antes de aplicar técnicas mais avançadas de estatística ou aprendizagem de máquina. O objetivo da análise exploratória de dados é obter uma compreensão mais profunda dos dados, identificar padrões, tendências e insights que possam orientar a tomada de decisão e a solução de problemas. A análise exploratória de dados pode ser realizada de forma visual, usando gráficos, tabelas e diagramas, ou de forma numérica, usando medidas de tendência central, dispersão, correlação e teste de hipóteses. A escolha dos métodos depende do tipo e da qualidade dos dados disponíveis, bem como da pergunta ou objetivo que se quer responder com a anál

<!-- <hr style="border-width: 1px" width="95%" > -->
<div></div> 

In [1]:
# Importa os módulos necessários
import numpy as np   # Módulo para trabalhar com matrizes e funções matemáticas
import pandas as pd  # Módulo para trabalhar com dataframes e séries em Python


<div></div> 

## Estruturação dos Arquivos

<div></div> 

In [2]:
base = pd.read_parquet('../data/processed/base_processed.parquet.gzip')

In [19]:
import itertools
import collections

counts_no = collections.Counter(list(itertools.chain(*base['tokens'])))

counts_no.most_common(30)

[('edu', 72452),
 ('ax', 47799),
 ('c', 27223),
 ('wa', 24454),
 ('line', 23074),
 ('com', 22785),
 ('message', 22263),
 ('subject', 21661),
 ('news', 21081),
 ('date', 20828),
 ('id', 20598),
 ('newsgroups', 20363),
 ('path', 20353),
 ('cmu', 20143),
 ('organization', 20077),
 ('apr', 19593),
 ('gmt', 17725),
 ('one', 16548),
 ('would', 15997),
 ('srv', 15120),
 ('cantaloupe', 15069),
 ('writes', 14685),
 ('1', 13931),
 ('article', 13867),
 ('reference', 13740),
 ('ha', 13217),
 ('net', 13057),
 ('x', 12252),
 ('state', 11585),
 ('sender', 11024)]